# HuggingFace smolagents Practice
This notebook demonstrates how to use the `smolagents` framework to create a basic agent, define custom tools, integrate MCP tools, and observe the agent's behavior.

We will use the **GPT-4.1-mini** model for this tutorial.

## 1. Install and Import Dependencies
First, we install the `smolagents` library and `litellm` which allows us to use various LLM providers. We also need `openai` for GPT-4.1-mini.

In [3]:
%pip install smolagents litellm openai ddgs

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from smolagents import CodeAgent, LiteLLMModel, Tool, tool

## 2. Configure the OpenAI Model
We will use `LiteLLMModel` to connect to OpenAI's `gpt-4.1-mini`.
**Note:** You need to set your OpenAI API key and base URL.

In [3]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

if "OPENAI_BASE_URL" not in os.environ:
    os.environ["OPENAI_BASE_URL"] = getpass.getpass("Enter your OpenAI Base URL: ")

model = LiteLLMModel(
    model_id="gpt-4.1-mini",
    api_key=os.environ["OPENAI_API_KEY"],
    api_base=os.environ["OPENAI_BASE_URL"]
)

Enter your OpenAI API Key: ··········
Enter your OpenAI Base URL: ··········


## 3. Create a Basic CodeAgent
We'll start by creating a simple `CodeAgent` without any custom tools. This agent can write and execute Python code to solve problems.

In [4]:
agent = CodeAgent(tools=[], model=model)
agent.run("Calculate the 10th Fibonacci number.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the 10th Fibonacci number.                                                                            │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  def fibonacci(n):                                                                                                
      a, b = 0, 1                                                                                                  
      for _ in range(n-1):                                                                                         
          a, b = b, a + b                                                                                          
      return a                                                                                                     
                                                                                                                   
  fib_10 = fibonacci(10)                                                                                           
  final_answer(fib_10)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 34

[Step 1: Duration 2.89 seconds| Input tokens: 1,977 | Output tokens: 100]

34

## 4. Define a Custom Tool
We can extend the agent's capabilities by defining custom tools. Here, we create a simple calculator tool using the `@tool` decorator.

In [5]:
@tool
def calculate_tip(bill_amount: float, tip_percentage: float = 15.0) -> str:
    """
    Calculates the tip amount and total bill.

    Args:
        bill_amount: The total cost of the meal.
        tip_percentage: The percentage of tip to leave (default is 15%).
    """
    tip = bill_amount * (tip_percentage / 100)
    total = bill_amount + tip
    return f"Tip: ${tip:.2f}, Total: ${total:.2f}"

# Test the tool directly
print(calculate_tip(100, 20))

Tip: $20.00, Total: $120.00


## 5. Integrate an MCP Tool
The Model Context Protocol (MCP) allows agents to connect to external tools standardized across different systems.

In `smolagents`, you can load tools from an MCP server.
```python
# Example of loading from an MCP server (requires a running MCP server)
# from smolagents import Tool
# mcp_tool = Tool.from_mcp("http://localhost:8000/mcp")
```

For this practice, we will simulate an MCP tool that retrieves system status.

In [6]:
@tool
def get_system_status(system_id: str) -> str:
    """
    Retrieves the status of a specific system.

    Args:
        system_id: The ID of the system to check (e.g., 'server-01').
    """
    # Simulated MCP response
    statuses = {"server-01": "Online", "server-02": "Maintenance", "db-prod": "Active"}
    return f"System {system_id} status: {statuses.get(system_id, 'Unknown')}"

## 6. Enable Observability
Observability is crucial for understanding how your agent makes decisions. `smolagents` provides built-in logging.
We can also use `GradioUI` for an interactive trace, but here we will rely on the console output which shows the agent's "thought" process.

When running the agent, `smolagents` prints the steps:
1.  **Thought**: The agent's reasoning.
2.  **Code**: The code generated to call a tool.
3.  **Observation**: The result from the tool.

In [7]:
# No specific code needed to enable basic logging, it's on by default in the run output.
# However, we can inspect the agent's logs after execution.
print("Observability is enabled by default. Watch the output of the next cell.")

Observability is enabled by default. Watch the output of the next cell.


## 7. Run the Agent with Tools
Now we combine everything. We create a new `CodeAgent` equipped with both our `calculate_tip` tool and the `get_system_status` tool.
We will give it a task that requires using both.

In [8]:
# Initialize agent with tools
final_agent = CodeAgent(
    tools=[calculate_tip, get_system_status],
    model=model,
    add_base_tools=True # Adds basic tools like search if available/configured, or just python execution
)

# Run with a complex query
response = final_agent.run(
    "I had a meal costing $150. Calculate the tip at 20%. "
    "Also, check the status of 'server-01' to see if I can pay online."
)

print(f"Final Answer: {response}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I had a meal costing $150. Calculate the tip at 20%. Also, check the status of 'server-01' to see if I can pay  │
│ online.                                                                                                         │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1-mini ───────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  tip_result = calculate_tip(bill_amount=150, tip_percentage=20)                                                   
  print(tip_result)                                                                                                
  server_status = get_system_status(system_id="server-01")                                                         
  print(server_status)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Tip: $30.00, Total: $180.00
System server-01 status: Online

Out: None

[Step 1: Duration 2.26 seconds| Input tokens: 2,214 | Output tokens: 98]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer({"tip_amount": 30.00, "total_bill": 180.00, "server_status": "Online", "can_pay_online": True})     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: {'tip_amount': 30.0, 'total_bill': 180.0, 'server_status': 'Online', 'can_pay_online': True}

[Step 2: Duration 1.41 seconds| Input tokens: 4,648 | Output tokens: 190]

Final Answer: {'tip_amount': 30.0, 'total_bill': 180.0, 'server_status': 'Online', 'can_pay_online': True}


## Conclusion
You have successfully created a `smolagents` agent powered by Gemini 2.5 Flash, defined custom tools, simulated an MCP tool integration, and observed the agent's execution flow.
Experiment with adding more tools or changing the model to see how the agent adapts!

## 8. Challenge: Build a Smart Home Controller
Now it's your turn! Apply what you've learned to build a simple smart home controller.

**Your Task:**
1.  Create a custom tool `get_room_temperature(room: str)` that returns a simulated temperature (e.g., "18°C").
2.  Create a custom tool `set_thermostat(room: str, target_temp: int)` that simulates setting the temperature and returns a confirmation message.
3.  Instantiate a `CodeAgent` with these tools and the Gemini model.
4.  Run the agent with a query: *"Check the temperature in the Living Room. If it's below 20°C, set the thermostat to 22°C."*

In [ ]:
# Implement your solution here
# 1. Define get_room_temperature tool

# 2. Define set_thermostat tool

# 3. Create the agent

# 4. Run the agent